In [1]:
import os
import gc

import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 200)

%load_ext autoreload
%autoreload 2

In [2]:
data_dir = '../data'
data_filepath = os.path.join(data_dir, 'harddrive.csv')
os.path.exists(data_filepath)

True

## Load data with PySpark session

In [3]:
spark = (SparkSession.builder
         .config("spark.driver.memory", "12g")
         .appName("SparkSQL").getOrCreate())

your 131072x1 screen size is bogus. expect trouble
24/08/23 16:10:36 WARN Utils: Your hostname, cedric-yu-work resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/08/23 16:10:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/23 16:10:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
date_col = 'date'
target_label = 'failure'
id_cols = ['serial_number', 'model']


In [5]:
df = (spark.read.option("header", "true")
      .option("inferSchema", "true")
      .csv(data_filepath))

In [6]:
df.printSchema()

root
 |-- date: date (nullable = true)
 |-- serial_number: string (nullable = true)
 |-- model: string (nullable = true)
 |-- capacity_bytes: double (nullable = true)
 |-- failure: integer (nullable = true)
 |-- smart_1_normalized: integer (nullable = true)
 |-- smart_1_raw: integer (nullable = true)
 |-- smart_2_normalized: string (nullable = true)
 |-- smart_2_raw: string (nullable = true)
 |-- smart_3_normalized: integer (nullable = true)
 |-- smart_3_raw: integer (nullable = true)
 |-- smart_4_normalized: integer (nullable = true)
 |-- smart_4_raw: integer (nullable = true)
 |-- smart_5_normalized: integer (nullable = true)
 |-- smart_5_raw: integer (nullable = true)
 |-- smart_7_normalized: integer (nullable = true)
 |-- smart_7_raw: double (nullable = true)
 |-- smart_8_normalized: string (nullable = true)
 |-- smart_8_raw: string (nullable = true)
 |-- smart_9_normalized: integer (nullable = true)
 |-- smart_9_raw: integer (nullable = true)
 |-- smart_10_normalized: integer (nul

In [7]:
df.count()

3179295

In [8]:
df1 = df.dropDuplicates([date_col] + id_cols)

In [9]:
df1.show(10)

24/08/23 15:57:39 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+-------------+--------------------+-------------------+-------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+---

In [10]:
df1.write.csv(os.path.join(data_dir, 'harddrive_nodup'), header=True)

In [10]:
gc.collect()

161

In [11]:
print(df.agg({date_col: "min"}).collect()[0])
print(df.agg({date_col: "max"}).collect()[0])

Row(min(date)=datetime.date(2016, 1, 1))


Row(max(date)=datetime.date(2016, 4, 29))


In [45]:
df.agg(F.countDistinct(df[target_label])).collect()

[Row(count(DISTINCT failure)=2)]

In [18]:
df.groupby([target_label]).count().show()

+-------+-------+
|failure|  count|
+-------+-------+
|      1|    215|
|      0|3179080|
+-------+-------+



In [46]:
df.agg(F.countDistinct(df['model'])).collect()

[Row(count(DISTINCT model)=69)]

In [23]:
df.select('model').distinct().show(10)

+--------------------+
|               model|
+--------------------+
|         ST9250315AS|
|         ST4000DM000|
|          ST320LT007|
|         ST3500320AS|
|      WDC WD5002ABYS|
|      WDC WD2500BEVT|
|Hitachi HDS5C3030...|
|        WDC WD60EFRX|
|       WDC WD800AAJB|
|Hitachi HDS722020...|
+--------------------+
only showing top 10 rows



In [47]:
df.agg(F.countDistinct(df['serial_number'])).toPandas()

,count(DISTINCT serial_number)
0,65993


In [25]:
df.select('serial_number').distinct().show(10)

+---------------+
|  serial_number|
+---------------+
|       S300Z60P|
|WD-WMAYUU472236|
| PL2331LAGMNX6J|
| S2ZYJ9BF508152|
|       Z300CNRB|
| PL2331LAGU9EYJ|
| PL1331LAGSAE0H|
|       Z305B6JC|
| JK11A8B9J7AAKF|
| JK11A8B9J7W9AF|
+---------------+
only showing top 10 rows



In [50]:
df.where(df[target_label] == 1).agg(F.countDistinct('serial_number')).show()

+-----------------------------+
|count(DISTINCT serial_number)|
+-----------------------------+
|                          205|
+-----------------------------+



In [51]:
df.groupby(id_cols).count().sort('count', ascending=False).show(10)

+---------------+--------------------+-----+
|  serial_number|               model|count|
+---------------+--------------------+-----+
| MJ1311YNG7BH3A|Hitachi HDS5C3030...|   52|
|       S300Z6K9|         ST4000DM000|   52|
|       S300ZVMZ|         ST4000DM000|   52|
|       S3010TR7|         ST4000DM000|   52|
| PL2331LAGM7YGJ|HGST HMS5C4040BLE640|   52|
|       S30088X7|         ST4000DM000|   52|
|       Z305BT2T|         ST4000DM000|   52|
| PL1331LAGT94SH|HGST HMS5C4040ALE640|   52|
|WD-WX41D94RNV69|        WDC WD60EFRX|   52|
| JK11A8B9J9KYXF|Hitachi HDS722020...|   52|
+---------------+--------------------+-----+
only showing top 10 rows



In [11]:
spark.stop()